# Word2Vec + TF-IDF 

這篇的資料是用 rawcut 哦

一共分為六個部分：
> 1. 資料預處理
> 2. word2vec model
> 3. TF-IDF 加權平均詞向量
> 4. 檢測 w2v
> 6. 檢測 tfidf
> 5. 檢測 w2v+tfidf

[參考文章 01 ](https://collabin.netlify.app/derek/title-clustering/)  
[參考文章 01-2](https://collabin.netlify.app/derek/title-clustering-tfidf/)

In [1]:
! pip install gensim
! pip install openpyxl

## 1. 資料預處理

In [234]:
import pandas as pd
import glob
import numpy as np
#import re
#from pandas import Series

# 匯入檔案
files = glob.glob("D:/大學/專題/AllData/rawdata/05summary_raw/*_w2v.xlsx")
#files = glob.glob("D:/大學/專題/AllData/rawdata/04bystop_raw/raw_c500.xlsx")
#files = glob.glob("D:/大學/專題/AllData/alldata_test.csv")
#用csv進來 下面pd.read_excel記得要改成pd.read_csv
files

['D:/大學/專題/AllData/rawdata/05summary_raw\\Belle_w2v.xlsx',
 'D:/大學/專題/AllData/rawdata/05summary_raw\\Fu_w2v.xlsx',
 'D:/大學/專題/AllData/rawdata/05summary_raw\\Liang_w2v.xlsx',
 'D:/大學/專題/AllData/rawdata/05summary_raw\\Oldyi_w2v.xlsx',
 'D:/大學/專題/AllData/rawdata/05summary_raw\\Peng_w2v.xlsx',
 'D:/大學/專題/AllData/rawdata/05summary_raw\\rebecca2_w2v.xlsx']

In [235]:
# 匯入檔案
file2 = glob.glob("D:/大學/專題/AllData/emojidata/summary/icon49_*.xlsx")
#file2 = glob.glob("D:/大學/專題/AllData/emojidata/emojistopwords/icon_nsw_c500.xlsx")
#glob.glob("D:/大學/專題/AllData/emojidata/emojistopwords/01/icon_nsw_c*.xlsx")
file2

['D:/大學/專題/AllData/emojidata/summary\\icon49_Peng.xlsx',
 'D:/大學/專題/AllData/emojidata/summary\\icon49_大福.xlsx',
 'D:/大學/專題/AllData/emojidata/summary\\icon49_柯南.xlsx',
 'D:/大學/專題/AllData/emojidata/summary\\icon49_歐易.xlsx',
 'D:/大學/專題/AllData/emojidata/summary\\icon49_貝卡.xlsx',
 'D:/大學/專題/AllData/emojidata/summary\\icon49_貝爾.xlsx']

In [237]:
#%%time
from math import sqrt
import time

time_start = time.time()

# 合併所有 Cut 
df = pd.concat(
    (pd.read_excel(file, usecols=['NoStopCut'], dtype={'NoStopCut': str}) for file in files), ignore_index=True)
print("總共有 "+str(len(df))+" 筆資料")

# 匯出檔案 --也可以選擇不匯出哦
#df.to_excel("D:/大學/專題/AllData/word2vec/" + "AllNoStopCut.xlsx", header = True, index = False)
#print("檔案已匯出")

# 語料庫 
# 轉成進 w2v 和 w2v&tfidf的形式
corpus = []
cutlist = df['NoStopCut'].tolist()

for i in range(len(cutlist)):
    corpus.append(eval(cutlist[i]))
len(corpus)

time_end = time.time()
timespend = (time_end- time_start)/60


print("語料庫完成，可以繼續下一步了哦 ✧*｡٩(ˊᗜˋ*)و✧*｡ ")
print("時間: ", round(timespend, 2), " 分鐘")

總共有 2881927 筆資料
語料庫完成，可以繼續下一步了哦 ✧*｡٩(ˊᗜˋ*)و✧*｡ 
時間:  9.9  分鐘


In [ ]:
%%time
# 合併所有 Cut 
df = pd.concat(
    (pd.read_excel(file, usecols=['NoStopCut','emoji','Group'
                                 ],
                   dtype={'NoStopCut': str,'emoji': str
                          ,'Group': str
                         }) for file in file2), ignore_index=True)
print("總共有 "+str(len(df))+" 筆資料")

# 用空格替代標點符號
df['NoStopCut'] = df['NoStopCut'].str.replace('\'|\[|\]|,','')


##整理成w2v要的形式
emojidata=[]
df['try'] = 0

for i in range (df.shape[0]):
    #df['try'][i] = '[\''+df['NoStopCut'][i]+' '+df['emoji'][i]+'\']'
    df['try'][i] = '[\''+df['NoStopCut'][i]+'\']'
df['try'] = df['try'].str.replace(' ','\',\'') 
#df['try'].to_excel("D:/大學/專題/AllData/word2vec/" + "emojitry.xlsx", header = True, index = False)

##合併
cutlist = df['try'].tolist()

for i in range(len(cutlist)):
    emojidata.append(eval(cutlist[i]))
len(emojidata)
print("表情語料庫完成，可以繼續下一步了哦 ✧*｡٩(ˊᗜˋ*)و✧*｡ ")


##整理成tfidf要的形式
emojidata2=[]
df['try'] = 0

for i in range (df.shape[0]):
    #df['try'][i] = '[\''+df['NoStopCut'][i]+' '+df['emoji'][i]+'\']'
    df['try'][i] = '\''+df['NoStopCut'][i]+'\''
#df['try'].to_excel("D:/大學/專題/AllData/word2vec/" + "emojitry.xlsx", header = True, index = False)

##合併
cutlist2 = df['try'].tolist()

for i in range(len(cutlist2)):
    emojidata2.append(eval(cutlist2[i]))
len(emojidata2)
print("表情語料庫完成，可以繼續下一步了哦 ✧*｡٩(ˊᗜˋ*)و✧*｡ ")

#df.to_excel("D:/大學/專題/AllData/emojidata/summary/" + "Allemoji.xlsx", header = True, index = False)
#print("檔案已匯出")

## 2. w2v model
用 gensism 的 w2v 構建詞向量

In [ ]:
%%time
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from gensim.models.callbacks import CallbackAny2Vec
import gensim
import pickle 

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''
        
    def __init__(self):
        self.epoch = 1
        self.losses = []
        self.cumu_loss = 0.0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        self.cumu_loss += float(loss)
        print(f"Loss after epoch {self.epoch}: {loss} / cumulative loss: {self.cumu_loss}")
        self.epoch += 1
        self.losses.append(float(loss))
        model.running_training_loss = 0.0

w2v_model = gensim.models.word2vec.Word2Vec(
    sentences = corpus, 
    min_count = 5, #若詞頻小於 min_count，那他就不會被視為訓練對象
    max_vocab_size = None, #詞典容納上限，出現次數最低的詞會優先被剔除
    vector_size = 300, #建議 250到 400之間
    workers=4, # 可以加快訓練速度，但除非電腦不錯，不然建議不要超過四 （網上是說要先安裝Cython，不然只能單核）
    epochs = 30, #迭代次數
    window = 9, #windowsize
    sg = 0, # sg=1->skip-gram; sg=0->cbow 
    #hs=0 # hs=0->nagative samling; h2=1->hierarchical softmax
    compute_loss = True,
    callbacks = [callback()]
)

print(w2v_model)

# 模型保存 ———— 模型會被保存在和這篇程式碼一樣的的資料夾下
w2v_model.save('word2vecold.model') #可繼續在此基礎上接著訓練
w2v_model.wv.save_word2vec_format('word2vecold.vector', binary=False) #可開啟檢視；不能繼續訓練

In [ ]:
#原有基礎上訓練新資料
model = gensim.models.Word2Vec.load('word2vecold.model')
model.build_vocab(emojidata2, update=True)
model.train(emojidata, total_examples=model.corpus_count, epochs=model.epochs)

model.save('word2vec.model') #可繼續在此基礎上接著訓練
model.wv.save_word2vec_format('word2vec.vector', binary=False) #

In [ ]:
wdf = pd.DataFrame(model, columns = ['詞','詞向量'])
wdf1 = pd.DataFrame(emojidata2,columns=['句'])

In [ ]:
# 匯入pickle
with open('word2vec_model.pickle','wb') as f:
    pickle.dump(w2v_model,f)
# pickle和vector的檔案已經有包含對應的vocab和向量

#-----------到這裡已經有訓練好的詞向量了------------------
# 接著我們就用 tfidf 加權 w2v向量 句向量的關鍵特征

## 3. TF-IDF 加權平均詞向量
對詞求 TF-IDF , 然後乘以相應的權重取得句向量

In [45]:
import numpy as np

#tfidf 加權平均詞向量
def tfidf_wtd_avg_word_vectors(words, tfidf_vector, 
                               tfidf_vocabulary, model, num_features):
    word_tfidfs = [tfidf_vector [0, tfidf_vocabulary.get(word)] 
                       if tfidf_vocabulary.get(word) else 0 for word in words]
    word_tfidf_map = {word : tfidf_val for word, 
                          tfidf_val in zip(words, word_tfidfs)}
    feature_vector = np.zeros((num_features,), dtype = 'float64') #依佔據內存大小排：float64、float32、float16、unit8
    vocabulary = set(model.wv.index_to_key)
    wts = 0
    
    # 得出各個詞的特征向量，並形成一個矩陣，然後計算平均值就得到該句子的特征向量
    for word in words:
        if word in vocabulary:
            word_vector = model.wv[word]
            weighted_word_vector = word_tfidf_map[word] * word_vector
            wts = wts + word_tfidf_map[word]
            feature_vector = np.add(feature_vector,weighted_word_vector)
    if wts:
        feature_vector = np.divide(feature_vector,wts)
    return feature_vector


def tfidf_weighted_averaged_word_vectorizer(rcorpus, tfidf_vectors, tfidf_vocabulary, model, num_features):
    docs_tfidfs = [(doc, doc_tfidf) for doc, 
                   doc_tfidf in zip(rcorpus,tfidf_vectors)]
    features = [tfidf_wtd_avg_word_vectors(tokenized_sentence, 
                                           tfidf, tfidf_vocabulary, 
                                           model, num_features) 
                for tokenized_sentence,tfidf in docs_tfidfs]
    return np.array(features)


#### TFIDF 預處理 

In [50]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

def tfidf_transformer(bow_matrix):   
    transformer = TfidfTransformer(norm='l2',
                                   smooth_idf=True,
                                   use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix) 
    return transformer, tfidf_matrix

# TfidfVectorizer 是 CountVectorizer和 TfidfTransformer的結合體，
# 用於統計 vectorizer中每個詞的 TFIDF值
def tfidf_extractor(rcorpus, ngram_range=(1,1)):  
    vectorizer = TfidfVectorizer(token_pattern='\\b\\w+\\b',norm='l2',smooth_idf=True, use_idf=True,ngram_range=ngram_range)
    features = vectorizer.fit_transform(rcorpus) # fit是學習idf_vector，transform是把計數矩陣轉換成td_idf表示
    return vectorizer, features

# CountVectorizer會將文本中的詞語轉換為詞頻矩陣，
# 通過 fit_transform 計算各個詞語出現的次數.
def bow_extractor(rcorpus, ngram_range=(1,1)):
    # min_df为1说明文档中词频最小为1也会被考虑
    # ngram_range可以设置(1,3)将建立包括所有unigram、bigram、trigram的向量空间
    ctorizer = CountVectorizer(token_pattern='\\b\\w+\\b',ngram_range=ngram_range)
    features = vectorizer.fit_transform(rcorpus) # 根據語料庫統計詞袋(fit)，並轉化為詞袋向量(transform)
    return vectorizer, features

#### 輸出詞的Tf-Idf值 

In [141]:
%%time
bow_vectorizer, bow_features = bow_extractor(emojidata2) # 根據我們的語料庫通詞袋
feature_names = bow_vectorizer.get_feature_names() # 獲得所有文本的關鍵字

tfidf_trans, tfidf_features = tfidf_transformer(bow_features) #將上面的語料庫轉換為稀疏矩陣
tfidf_vectorizer, tdidf_features = tfidf_extractor(emojidata2)
feature_names2 = tfidf_vectorizer.get_feature_names() # 獲得所有文本的關鍵字

#emojitfidf = tfidf_vectorizer.transform(emojidata)

Wall time: 14 ms


In [ ]:
# (0,2107) 0.456 => 0是第一句子，2107是詞在語料庫裡面的ID, 0.456是 tfidf
# 這裡的語料庫 ID 和 word2vec的語料庫 

print(tdidf_features) # tdidf的特征值

df_word_tfidf = pd.DataFrame(list(zip(emojidata,tdidf_features)),columns=['詞','TFIDF'])

df_word_tfidf
#df_word_tfidf.to_excel("D:/大學/專題/AllData/rawdata/summary_raw/" + "TfidfT.xlsx", header = True, index = False)

#---------這裡可以得到每個單詞相對於各個句子的權重

In [78]:
for idx, word in enumerate(tfidf_vectorizer.get_feature_names()):
    print("{}\t{}".format(word, tfidf_vectorizer.idf_[idx]))

一	3.6729588812909397
一路	4.401197381662156
三小	6.3471075307174685
上演	6.3471075307174685
不對勁	6.3471075307174685
不用	5.941642422609304
不行	5.653960350157523
不過	6.3471075307174685
中共	6.3471075307174685
中國人	5.941642422609304
主	6.3471075307174685
主動	6.3471075307174685
事	5.941642422609304
互稱	6.3471075307174685
人生	5.941642422609304
人群	5.941642422609304
代	6.3471075307174685
代替	6.3471075307174685
代表	6.3471075307174685
伯恩	6.3471075307174685
位	5.248495242049359
位置	5.430816798843313
低	6.3471075307174685
住	6.3471075307174685
作	6.3471075307174685
作者	6.3471075307174685
來得	6.3471075307174685
保安	6.3471075307174685
保證	6.3471075307174685
信誓旦旦	6.3471075307174685
修真	6.3471075307174685
修美	6.3471075307174685
倒黴	6.3471075307174685
假	6.3471075307174685
做	5.653960350157523
做成	6.3471075307174685
偷來	6.3471075307174685
偷吃	6.3471075307174685
偷走	6.3471075307174685
傻眼	5.941642422609304
優秀	5.941642422609304
先生氣	6.3471075307174685
光明磊落	6.3471075307174685
兒子	5.941642422609304
兒童	6.3471075307174685
內文	6.3471075307174685
內衣	6

親愛	5.941642422609304
親愛的	5.430816798843313
言	6.3471075307174685
討厭	5.941642422609304
討論到	6.3471075307174685
記得	6.3471075307174685
記者	6.3471075307174685
話	5.941642422609304
認愛	5.941642422609304
語	6.3471075307174685
語錄	6.3471075307174685
誤會	6.3471075307174685
說	4.332204510175204
說法	6.3471075307174685
說瞎話	6.3471075307174685
說話	6.3471075307174685
請	6.3471075307174685
請問	6.3471075307174685
講	6.3471075307174685
講出來	6.3471075307174685
講話	5.653960350157523
謝先生	6.3471075307174685
證據	6.3471075307174685
貓	5.941642422609304
貝薇雅	6.3471075307174685
貨	6.3471075307174685
買	6.3471075307174685
貼上	6.3471075307174685
貼切	6.3471075307174685
貼紙	6.3471075307174685
賣	6.3471075307174685
赦免	6.3471075307174685
走	4.267665989037633
起步	6.3471075307174685
起訴不了	6.3471075307174685
超	5.941642422609304
超生	6.3471075307174685
超級	5.941642422609304
跑	6.3471075307174685
踩	6.3471075307174685
身影	6.3471075307174685
車車	6.3471075307174685
軟爛	6.3471075307174685
轉	6.3471075307174685
辦	6.3471075307174685
退出	6.3471075307174685
連續劇	6.3

In [171]:
#f1 = pd.read_excel("D:/大學/專題/AllData/emojidata/summary/01/tfidf.xlsx", usecols=['詞','TFIDF'], 
 #                 dtype={'詞': str,'TFIDF': str}) 

df3 = pd.read_excel("D:/大學/專題/AllData/emojidata/summary/01/Allemoji.xlsx", usecols=['emoji','Group'], 
                   dtype={'emoji': str,'Group': str}) 


In [ ]:
vocob=tfidf_vectorizer.vocabulary_
vocob
print (len(vocob))

In [184]:
dense = tdidf_features.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns = feature_names2)
df1 = pd.DataFrame(emojidata2,columns=['句'])
df2 = pd.concat([df3,df1,df],axis=1)
df2.to_excel("D:/大學/專題/AllData/emojidata/summary/01/" + "tfidf3.xlsx", header = True, index = False)

In [187]:
pickle.dump(df2, open("tfidf.pickle","wb")) #可繼續在此基礎上接著訓練

In [ ]:
with open("tfidf.pickle","rb") as f:
    modelP = pickle.load(f)
print(modelP) #印出全部
#print(modelP.詞) #印出詞
#print(modelP.tfidf) #印出 w2vtfidf的向量

#-----------到這裡已經有訓練好的tfidf了------------------
# 接著我們就用 tfidf 加權 w2v向量 句向量

## 輸出w2v+TFIDF向量 ---變成句向量 
(檔案大的話 這裡會跑蠻久的)

 > 以每個句子的 tfidf 向量作為加權乘上 w2v 中所有詞的詞向量來產出一個包含所有句子向量的矩陣（這樣一來每一個句子更能包含一個詞語的重要性）

#### TFIDF加權詞向量

In [189]:
corpus_tfidf = tfidf_features #詞向量
vocab = tfidf_vectorizer.vocabulary_ #詞表

In [16]:
#corpus_tfidf
#type(corpus_tfidf)
#print(corpus_tfidf)

In [223]:
import pickle

time_start = time.time()

wt_tfidf_word_vec_features=tfidf_weighted_averaged_word_vectorizer(emojidata, 
                                                                   tfidf_vectors=corpus_tfidf,
                                                                       tfidf_vocabulary=vocab,
                                                                   model=w2v_model,
                                                                   num_features=300)

print(wt_tfidf_word_vec_features)


# 模型保存 ———— 模型會被保存在和這篇程式碼一樣的的資料夾下
pickle.dump(wt_tfidf_word_vec_features, open("w2vtfidf_vector.pickle","wb")) #可繼續在此基礎上接著訓練


# 這個是把模型叫出來 （下面檢視 w2v+tfidf的時間才叫）
#wt_tfidf_word_vec_features = pickle.load(open("vectorizer.pickle","rb"))

time_end = time.time()
timespend = (time_end- time_start)/60

print("恭喜 w2v+tfidf 完成嘍  ୧༼◕ ᴥ ◕༽୨ ")
print("時間: ", round(timespend, 2), " 分鐘")

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00534932  0.13601139 -0.00888613 ... -0.16714188  0.08900192
  -0.08928172]
 ...
 [ 0.0051786   0.13760675 -0.00739777 ... -0.16489953  0.09335098
  -0.08251653]
 [ 0.00737089  0.14362137 -0.00788148 ... -0.17586666  0.10247523
  -0.09551062]
 [ 0.00413144  0.03599213 -0.00381419 ... -0.0470253   0.02774737
  -0.02179987]]
恭喜 w2v+tfidf 完成嘍  ୧༼◕ ᴥ ◕༽୨ 
時間:  0.0  分鐘


## w2v+tfidf 檔案匯出

In [218]:
# 還要加句子和 emoji

w2vtfidf_doc = pd.DataFrame(list(zip(emojidata, wt_tfidf_word_vec_features)), columns=['句','句向量'])
w2vtfidf_doc.to_excel("D:/大學/專題/AllData/emojidata/summary/01/" + "w2vtfidf.xlsx", header = True, index = False)

In [ ]:
wtdf1 = pd.read_excel("D:/大學/專題/AllData/emojidata/summary/01/w2vtfidf.xlsx", usecols=['詞','w2vtfidf'], 
                   dtype={'句': str,'句向量': str}) 

wtdf2 = pd.read_excel("D:/大學/專題/AllData/emojidata/summary/01/Allemoji.xlsx", usecols=['emoji','Group'], 
                   dtype={'emoji': str,'Group': str}) 

wtdf3 = pd.concat([wtdf2,wtdf1],axis=1)
wtdf3.to_excel("D:/大學/專題/AllData/emojidata/summary/" + "w2vtfidf.xlsx", header = True, index = False)

In [ ]:
pickle.dump(wtdf3, open("w2vtfidf.pickle","wb")) #可繼續在此基礎上接著訓練

In [ ]:
with open("w2vtfidf_vector.pickle","rb") as f:
    docV = pickle.load(f)
print(docV) #印出全部
#print(docV.句) #印出詞
#print(docV.句向量) #印出 w2vtfidf的向量

In [1]:
with open(r"C:\Users\e2414\OneDrive\桌面\專題\AllData\training\word2vec.pickle","rb") as f:
    w2vv = pickle.load(f)

NameError: name 'pickle' is not defined